In [1]:
import numpy as np
import csv
import random
import matplotlib.pyplot as plt

In [2]:
def sigmoid(row, weights):
    yhat = np.dot(row,weights)
    return 1/(1+np.exp(-yhat))    

In [3]:
def predict(val):
    return 1 if val >= 0.5 else 0

In [4]:
def accurac(weights,train,test) :
    correct = 0
    for i in range (len(train)) :
        if predict(sigmoid(train[i],weights)) == test[i] :
            correct = correct +1
    return (correct/(len(train)))*100
    

In [5]:
def rec_prec_f(weights,train,test,num) :
    trup=0
    prec=0
    rec=0
    fp=0
    fn = 0
    f=0
    for i in range (len(train)) :
        if test[i] == 1 :
            trup = trup + 1 
            if predict(sigmoid(train[i],weights)) != test[i] :
                fn = fn + 1
        if test[i] == 0 :
            if predict(sigmoid(train[i],weights)) != test[i] :
                fp = fp + 1
    prec = trup/(trup+fp)
    rec = trup/(trup+fn)
    f = 2*(prec*rec)/(prec+rec)
    if num == 0 : return rec*100  
    if num == 1 : return prec*100
    if num == 2 : return f

In [6]:
def loss(weights,test,labels) :
    loss = 0
    for i in range(len(test)) :
        yhat = sigmoid(weights,test[i])
        if yhat != 1 :
            loss = loss + labels[i]*np.log(yhat) + (1-labels[i])*np.log(1-yhat) 
    return -(loss/len(test))

In [13]:
def normalise(data) :
    mini = []
    maxi = []
    a1 = []
    a2 = []
    a3 = []
    a4 = []
    test = []
    for j in range(len(data)) :
        a1.append(data[j][0])
        a2.append(data[j][1])
        a3.append(data[j][2])
        a4.append(data[j][3])
    mini.append(min(a1))
    maxi.append(max(a1))
    mini.append(min(a2))
    maxi.append(max(a2))
    mini.append(min(a3))
    maxi.append(max(a3))
    mini.append(min(a4))
    maxi.append(max(a4))
    for i in range(len(data)) :
        row = []
        for j in range(len(data[0])-1) :
            row.append((data[i][j] - mini[j])/(maxi[j]-mini[j]))
        row.append(1)
        test.append(row)
    return test       

In [19]:
#model
filename = "dataset_LR.csv"
data = []
length = 5
with open(filename, 'r') as csvfile :
        csvreader = csv.reader(csvfile)
        i = 0
        for row in csvreader :
            if i ==0 :
                del row
                i = i+1
                continue
            data.append(row)
loss_1 = []
accuracy_1 = []
precision = []
recall = []
f_score = []
loss_2 = []
accuracy_2 = []
precision_1 = []
recall_1 = []
f_score_1 = []
for k in range(10) :
    test_labels = []
    train_labels = []
    test_data = []
    train_data = []
    random.shuffle(data)
    split = int(len(data)*0.7)
    for i in range(split) :
        train_data.append(data[i])
        train_labels.append(data[i][length-1])
        del train_data[i][length-1]
        train_data[i].append(1)
    for i in range(len(data)-split) :
        test_data.append(data[i+split])
        test_labels.append(data[i+split][length-1])
        del test_data[i][length-1]
        test_data[i].append(1)
    train_data = np.array(train_data).astype(np.float)
    test_data = np.array(test_data).astype(np.float)
    test_labels = np.array(test_labels).astype(np.int)
    train_labels = np.array(train_labels).astype(np.int)
    alpha = 0.1
    for m in range(2) :
        correction = 0
        loss_val = 0
        acc = 0
        prec = 0
        rec = 0
        f = 0
        weights = np.random.randn(length)
        if m == 0 :
            for i in range(20) :
                for j in range(len(train_data)) :
                    yhat = sigmoid(weights,train_data[j])
                    weights = weights - alpha*train_data[j]*(yhat - train_labels[j])
            acc = accurac(weights,test_data,test_labels)
            accuracy_1.append(acc)
            loss_val = loss(weights,test_data,test_labels)
            loss_1.append(loss_val)
            rec = rec_prec_f(weights,test_data,test_labels,0)
            recall.append(rec)
            prec = rec_prec_f(weights,test_data,test_labels,1)
            precision.append(prec)
            f = rec_prec_f(weights,test_data,test_labels,2)
            f_score.append(f)
            print("Accuracy for split",k+1,"for SGD is",acc)
            print("Loss for split",k+1,"for SGD is",loss_val)
            #print("\n")

        if m == 1 :
            for i in range(20) :
                for j in range(len(train_data)) :
                    yhat = sigmoid(weights,train_data[j])
                    correction = correction + train_data[j]*(yhat - train_labels[j])
                weights = weights - (alpha*correction)/len(train_data)
            acc = accurac(weights,test_data,test_labels)
            accuracy_2.append(acc)
            loss_val = loss(weights,test_data,test_labels)
            loss_2.append(loss_val)
            rec = rec_prec_f(weights,test_data,test_labels,0)
            recall_1.append(rec)
            prec = rec_prec_f(weights,test_data,test_labels,1)
            precision_1.append(prec)
            f = rec_prec_f(weights,test_data,test_labels,2)
            f_score_1.append(f)
            print("Accuracy for split",k+1,"for GD is",acc)
            print("Loss for split",k+1,"for GD is",loss_val)
            #print("\n")
    print("\n")
print("\n")
print("Average accuracy for SGD is",np.mean(accuracy_1))
print("Average accuracy for GD is",np.mean(accuracy_2))
print("\n")
print("Average loss for SGD is",np.mean(loss_1))
print("Average loss for GD is",np.mean(loss_2))
print("\n")
print("Average recall for SGD is",np.mean(recall))
print("Average recall for GD is",np.mean(recall_1))
print("\n")
print("Average precision for SGD is",np.mean(precision))
print("Average precision for GD is",np.mean(precision_1))
print("\n")
print("Average f-score for SGD is",np.mean(f_score))
print("Average f-score for SGD is",np.mean(f_score_1))

Accuracy for split 1 for SGD is 98.30097087378641
Loss for split 1 for SGD is 0.05308605161682734
Accuracy for split 1 for GD is 95.63106796116504
Loss for split 1 for GD is 0.20747516391762286


Accuracy for split 2 for SGD is 100.0
Loss for split 2 for SGD is 0.00042692447231611083
Accuracy for split 2 for GD is 90.0485436893204
Loss for split 2 for GD is 0.8283329686856121


Accuracy for split 3 for SGD is 100.0
Loss for split 3 for SGD is 0.0005271726909688486
Accuracy for split 3 for GD is 86.40776699029125
Loss for split 3 for GD is 0.8559754308159698


Accuracy for split 4 for SGD is 100.0
Loss for split 4 for SGD is 0.0004912737801445299
Accuracy for split 4 for GD is 89.80582524271846
Loss for split 4 for GD is 1.058753029434147


Accuracy for split 5 for SGD is 100.0
Loss for split 5 for SGD is 0.0005336467749378373
Accuracy for split 5 for GD is 99.27184466019418
Loss for split 5 for GD is 0.05204915059544937


Accuracy for split 6 for SGD is 100.0
Loss for split 6 for SGD i

In [18]:
#to plot graphs and calculate most important feature
train = normalise(train_data)
test = normalise(test_data)
alpha = 0.1
w1 = []
w2 = []
w3 = []
w4 = []
weights = np.random.randn(length)
accuracy_plot_1 = []
accuracy_plot_2 = []
epochs = []
for i in range(20) :
    for j in range(len(train_data)) :
        yhat = sigmoid(weights,train_data[j])
        weights = weights - alpha*train_data[j]*(yhat - train_labels[j])
    accuracy_plot_1.append(accurac(weights,test_data,test_labels))
    epochs.append(i+1)
    w1.append(weights[0])
    w2.append(weights[1])
    w3.append(weights[2])
    w4.append(weights[3])
weights = np.random.randn(length)
for k in range(20) :
    for j in range(len(train_data)) :
        yhat = sigmoid(weights,train_data[j])
        correction = correction + train_data[j]*(yhat - train_labels[j])
    weights = weights - (alpha*correction)/len(train_data)
    accuracy_plot_2.append(accurac(weights,test_data,test_labels))
#print(np.mean(w1))
#print(np.mean(w2))
#print(np.mean(w3))
#print(np.mean(w4))
#plt.plot(epochs,accuracy_plot_1, 'ro')
#plt.axis([1, 20, 0, 110])
#plt.show()

0.20293103895451794
0.40777022319053763
0.7045039469152703
-0.6287231035589947
